In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
##下面这几个在这里可有可有无，后续stacking集成的时候会用到
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor


C:\Users\zdq\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
# step1 检查源数据集，读入数据，将csv数据转换为DataFrame数据
train_df = pd.read_csv('./data.csv',index_col = 0)
test_df = pd.read_csv('./test.csv',index_col = 0)


In [4]:
# step2 合并数据，进行数据预处理
prices = pd.DataFrame({'price':train_df['SalePrice'],'log(price+1)':np.log1p(train_df['SalePrice'])})

In [5]:
y_train = np.log1p(train_df.pop('SalePrice'))
all_df = pd.concat((train_df,test_df),axis = 0)

In [7]:
# step3 变量转化
print(all_df['MSSubClass'].dtypes)
all_df['MSSubClass'] = all_df['MSSubClass'].astype(str)
print(all_df['MSSubClass'].dtypes)
print(all_df['MSSubClass'].value_counts())
# 把category的变量转变成numerical表达形式
# get_dummies方法可以帮你一键one-hot
print(pd.get_dummies(all_df['MSSubClass'],prefix = 'MSSubClass').head())
all_dummy_df = pd.get_dummies(all_df)
print(all_dummy_df.head())

int64
object
20     1079
60      575
50      287
120     182
30      139
160     128
70      128
80      118
90      109
190      61
85       48
75       23
45       18
180      17
40        6
150       1
Name: MSSubClass, dtype: int64
    MSSubClass_120  MSSubClass_150  MSSubClass_160  MSSubClass_180  \
Id                                                                   
1                0               0               0               0   
2                0               0               0               0   
3                0               0               0               0   
4                0               0               0               0   
5                0               0               0               0   

    MSSubClass_190  MSSubClass_20  MSSubClass_30  MSSubClass_40  \
Id                                                                
1                0              0              0              0   
2                0              1              0              0   
3    

In [9]:
# 处理好numerical变量
print(all_dummy_df.isnull().sum().sort_values(ascending = False).head(11))
# 我们这里用mean填充，也就是均值填充
mean_cols = all_dummy_df.mean()
print(mean_cols.head(10))
all_dummy_df = all_dummy_df.fillna(mean_cols)
print(all_dummy_df.isnull().sum().sum())


LotFrontage     486
GarageYrBlt     159
MasVnrArea       23
BsmtHalfBath      2
BsmtFullBath      2
BsmtFinSF2        1
GarageCars        1
TotalBsmtSF       1
BsmtUnfSF         1
GarageArea        1
BsmtFinSF1        1
dtype: int64
LotFrontage        69.305795
LotArea         10168.114080
OverallQual         6.089072
OverallCond         5.564577
YearBuilt        1971.312778
YearRemodAdd     1984.264474
MasVnrArea        102.201312
BsmtFinSF1        441.423235
BsmtFinSF2         49.582248
BsmtUnfSF         560.772104
dtype: float64
0


In [10]:
# 标准化numerical数据
numeric_cols = all_df.columns[all_df.dtypes != 'object']
print(numeric_cols)
numeric_col_means = all_dummy_df.loc[:,numeric_cols].mean()
numeric_col_std = all_dummy_df.loc[:,numeric_cols].std()
all_dummy_df.loc[:,numeric_cols] = (all_dummy_df.loc[:,numeric_cols] - numeric_col_means) / numeric_col_std


Index(['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')


In [11]:
# step4 建立模型
# 把数据处理之后，送回训练集和测试集
dummy_train_df = all_dummy_df.loc[train_df.index]
dummy_test_df = all_dummy_df.loc[test_df.index]
print (dummy_train_df.shape,dummy_test_df.shape)


(1460, 303) (1459, 303)


In [12]:
# 将DF数据转换成Numpy Array的形式，更好地配合sklearn

X_train = dummy_train_df.values
X_test = dummy_test_df.values


In [17]:
X_train.shape

(1460, 303)

In [18]:
X_test.shape

(1459, 303)

In [28]:
y=prices['price']

In [31]:
y

Id
1       208500
2       181500
3       223500
4       140000
5       250000
6       143000
7       307000
8       200000
9       129900
10      118000
11      129500
12      345000
13      144000
14      279500
15      157000
16      132000
17      149000
18       90000
19      159000
20      139000
21      325300
22      139400
23      230000
24      129900
25      154000
26      256300
27      134800
28      306000
29      207500
30       68500
         ...  
1431    192140
1432    143750
1433     64500
1434    186500
1435    160000
1436    174000
1437    120500
1438    394617
1439    149700
1440    197000
1441    191000
1442    149300
1443    310000
1444    121000
1445    179600
1446    129000
1447    157900
1448    240000
1449    112000
1450     92000
1451    136000
1452    287090
1453    145000
1454     84500
1455    185000
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: price, Length: 1460, dtype: int64

In [29]:
#step5 把训练集切分成两部分，80%训练，20%用作测试
import xgboost as xgb

from sklearn.metrics import mean_squared_error

import numpy as np

matrix=xgb.DMatrix(data=X_train,label=y)
from sklearn.model_selection import train_test_split

X_train1,X_test1,y_train1,y_test1=train_test_split(X_train,y,test_size=0.2,random_state=123)

In [33]:
X_train1.shape

(1168, 303)

In [35]:
X_test1.shape

(292, 303)

In [36]:
y_train1.shape

(1168,)

In [70]:
#step6 XGBoost模型和调参在这
xg_reg=xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.5,learning_rate=0.02,max_depth=4,alpha=8,n_estimators=800,subsample=0.7,random_state=123)
xg_reg.fit(X_train1,y_train1)
pred=xg_reg.predict(X_test1)
logrmse=np.sqrt(mean_squared_error(np.log(y_test1),np.log(pred)))

print(logrmse)

0.10348741097652386


In [71]:
xg_reg.fit(X_train, y)

XGBRegressor(alpha=8, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=800,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=123,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.7)

In [72]:
y_final = xg_reg.predict(X_test)

In [73]:
y_final

array([125820.89, 160634.14, 182633.25, ..., 157214.8 , 120481.92,
       222009.6 ], dtype=float32)

In [74]:
#step7 最后提交预测结果
submission_df = pd.DataFrame(data = {'Id':test_df.index,'SalePrice':y_final})
print (submission_df.head(10))
submission_df.to_csv('./submission_xgboost_2.csv',columns = ['Id','SalePrice'],index = False)


     Id      SalePrice
0  1461  125820.890625
1  1462  160634.140625
2  1463  182633.250000
3  1464  189562.359375
4  1465  186230.718750
5  1466  174786.015625
6  1467  168471.734375
7  1468  164855.500000
8  1469  181197.265625
9  1470  125115.890625


In [62]:
# # xgboost
# params = [1,2,3,4,5,6]
# test_scores = []
# for param in params:
#     clf = XGBRegressor(max_depth = param)
#     test_score = np.sqrt(-cross_val_score(clf,X_train,y_train,cv = 10,scoring = 'neg_mean_squared_error'))
#     test_scores.append(np.mean(test_score))
# plt.plot(params,test_scores)
# plt.title('max_depth vs CV Error')
# plt.show()




In [14]:
# xgb = XGBRegressor(max_depth = 5)
# xgb.fit(X_train, y_train)
# y_final = np.expm1(xgb.predict(X_test))

In [16]:
# submission_df = pd.DataFrame(data = {'Id':test_df.index,'SalePrice':y_final})
# print (submission_df.head(10))
# submission_df.to_csv('./submission_xgboosting.csv',columns = ['Id','SalePrice'],index = False)


     Id      SalePrice
0  1461  121317.992188
1  1462  160514.984375
2  1463  184872.671875
3  1464  188044.953125
4  1465  187002.328125
5  1466  176525.343750
6  1467  176312.000000
7  1468  163346.546875
8  1469  190007.890625
9  1470  122269.976562
